# The improper dihedral for benzene does not get written by ParmEd

In [1]:
%load_ext blackcellmagic

In [2]:
import numpy as np
import re

In [3]:
with open("../data/benzene.tripos.mol2") as file:
    benzenze = file.read()

Let's create an AMBER `frcmod` for reference:


```bash
antechamber -fi mol2 -i benzene.tripos.mol2 -fo mol2 -o benzene.gaff.mol2
tleap -f tleap.gaff.in
parmchk2 -i benzene.gaff.mol2 -f mol2 -o benzene.gaff.frcmod -s 2 -a Y
```

where `tleap.gaff.in` is:
```
source leaprc.gaff2
mol = loadmol2 benzene.gaff.mol2
saveamberparm mol benzene.gaff.prmtop benzene.gaff.inpcrd
```

The contents of the GAFF2 `frcmod` file is:

```
Remark line goes here
MASS
ca 12.010        0.360
ha 1.008         0.135

BOND
ca-ca  378.57   1.398
ca-ha  395.72   1.086

ANGLE
ca-ca-ca   68.767     120.020
ca-ca-ha   48.680     119.880

DIHE
ca-ca-ca-ca   4   14.500       180.000           2.000
ca-ca-ca-ha   4   14.500       180.000           2.000
ha-ca-ca-ha   4   14.500       180.000           2.000

IMPROPER
ca-ca-ca-ha         1.1          180.0         2.0          Using general improper torsional angle  X- X-ca-ha, penalty score=  6.0)

NONBON
  ca          1.8606  0.0988
  ha          1.4735  0.0161

```

# Right (OpenFF) workflow

Tripols MOL2 → OEMols → OpenMM system → MOL2, PRMTOP, FRCMOD (via ParmED)

In [1]:
from openeye.oechem import oemolistream, OEIFlavor_MOL2_Forcefield, OEFormat_MOL2, OEMol

In [2]:
from openforcefield.typing.engines.smirnoff import (
    ForceField,
    unit,
)
from openforcefield.topology import Molecule, Topology

In [3]:
import parmed as pmd

In [4]:
molecules = []
ifs = oemolistream()
ifs.SetFlavor(OEFormat_MOL2, OEIFlavor_MOL2_Forcefield)
ifs.open("../data/benzene.tripos.mol2")

for index, mol in enumerate(ifs.GetOEMols()):
    molecules.append(OEMol(mol))

In [7]:
[i for i in benzene_pdb.topology.bonds()]

[]

In [5]:
#off_molecule = Molecule.from_openeye(molecules[0])
off_molecule = Molecule.from_smiles('c1ccccc1')
print(off_molecule.n_atoms)
#topology = Topology.from_molecules([off_molecule])
benzene = pmd.load_file("../data/benzene.tripos.mol2")
#benzene.save("../data/benzene.pdb", overwrite=True)
from simtk.openmm.app import PDBFile
benzene_pdb = PDBFile("../data/benzene.pdb")
print(len([i for i in benzene_pdb.topology.atoms()]))

topology = Topology.from_openmm(benzene_pdb.topology, unique_molecules=[off_molecule])
ff = ForceField("smirnoff99Frosst.offxml")
system = ff.create_openmm_system(topology)

12
12


ValueError: No match found for molecule

Now, bootup ParmEd

12

In [66]:
parmed_structure = pmd.openmm.load_topology(benzene_pdb.topology, system)

In [74]:
for atom in benzene_pdb.topology.atoms():
    print(atom.name)

C
C1
C2
C3
C4
C5
H
H1
H2
H3
H4
H5


In [67]:
parmed_structure.save("../data/benzene.smirnoff.prmtop")

In [68]:
pmd.tools.writeFrcmod(pmd.load_file("../data/benzene.smirnoff.prmtop"), "../data/benzene.smirnoff.frcmod").execute()

ParameterError: Unequal dihedral types defined bewteen C1, C1, C1, and C1

In [45]:
from simtk.openmm import XmlSerializer

In [46]:
import pprint

In [51]:
pprint.pprint(XmlSerializer.serialize(system))

('<?xml version="1.0" ?>\n'
 '<System openmmVersion="7.3.1" type="System" version="1">\n'
 '\t<PeriodicBoxVectors>\n'
 '\t\t<A x="2" y="0" z="0"/>\n'
 '\t\t<B x="0" y="2" z="0"/>\n'
 '\t\t<C x="0" y="0" z="2"/>\n'
 '\t</PeriodicBoxVectors>\n'
 '\t<Particles>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t</Particles>\n'
 '\t<Constraints/>\n'
 '\t<Forces>\n'
 '\t\t<Force alpha="0" cutoff="1" dispersionCorrection="1" '
 'ewaldTolerance=".0005" forceGroup="0" ljAlpha="0" ljnx="0" ljny="0" ljnz="0" '
 'method="0" nx="0" ny="0" nz="0" recipForceGroup="-1" rfDielectric="78.3" '
 'switchingDistan

In [36]:
structure = pmd.openmm.load_topology(topology, system)

AttributeError: 'Topology' object has no attribute 'chains'

In [29]:
structure.save("benzene.smirnoff.prmtop")

NameError: name 'structure' is not defined

In [78]:
pmd.tools.writeFrcmod(pmd.load_file("phenol.prmtop"), "phenol.frcmod").execute()

In [39]:
topology.to_parmed()

NotImplementedError: 

In [1]:
dictionary = {'1': 'ca', '2': 'ca', '3': 'ca', '4': 'ca', '5': 'ca', '6': 'ca', '7': 'ha', '8': 'ha', '9': 'ha', '10': 'ha', '11': 'ha', '12': 'ha'}

In [25]:
with open("../benzene.gaff-smirnoff.frcmod", "r") as file:
    smirnoff_frcmod = file.read()

In [26]:
smirnoff_frcmod

'Force field parameters from benzene.gaff-smirnoff.prmtop\nMASS\n1     12.011\n2     12.011\n3     12.011\n4     12.011\n5     12.011\n6     12.011\n7      1.008\n8      1.008\n9      1.008\n10     1.008\n11     1.008\n12     1.008\n\nBOND\n1 -2     469.000   1.400\n1 -3     469.000   1.400\n2 -4     469.000   1.400\n3 -5     469.000   1.400\n4 -6     469.000   1.400\n5 -6     469.000   1.400\n1 -7     367.000   1.080\n2 -8     367.000   1.080\n3 -9     367.000   1.080\n4 -10    367.000   1.080\n5 -11    367.000   1.080\n6 -12    367.000   1.080\n\nANGLE\n2 -1 -3      70.000  120.000\n1 -2 -4      70.000  120.000\n1 -3 -5      70.000  120.000\n2 -4 -6      70.000  120.000\n3 -5 -6      70.000  120.000\n4 -6 -5      70.000  120.000\n2 -1 -7      50.000  120.000\n3 -1 -7      50.000  120.000\n1 -2 -8      50.000  120.000\n4 -2 -8      50.000  120.000\n1 -3 -9      50.000  120.000\n5 -3 -9      50.000  120.000\n2 -4 -10     50.000  120.000\n6 -4 -10     50.000  120.000\n3 -5 -11     50.00

In [40]:
import re
opening = "MASS"
between = "(.*?)"
closing = "BOND"
pattern = opening + between + closing
atoms = re.findall(pattern, smirnoff_frcmod, re.DOTALL)

In [60]:
atom_list = atoms[0].split("\n")

In [85]:
new_atoms = []
for atom in atom_list[1:-2]:
    atom_type, mass = atom.split()
    new_atoms.append(f"{dictionary[atom_type]}{mass:>10}")

1
2
3
4
5
6
7
8
9
10
11
12


In [87]:
opening = "BOND"
between = "(.*?)"
closing = "ANGLE"
pattern = opening + between + closing
bonds = re.findall(pattern, smirnoff_frcmod, re.DOTALL)

In [88]:
bond_list = bonds[0].split("\n")

In [132]:
new_bonds = []
for bond in bond_list[1:-2]:
    atom_one, atom_two, k, r = bond.split()
    atom_two = atom_two.replace("-", "")
    new_atom_one = dictionary[atom_one]
    new_atom_two = dictionary[atom_two]
    new_bonds.append(f"{new_atom_one:<2}-{new_atom_two:<2}{k:>12}{r:>7}")

In [133]:
bond_list

['',
 '1 -2     469.000   1.400',
 '1 -3     469.000   1.400',
 '2 -4     469.000   1.400',
 '3 -5     469.000   1.400',
 '4 -6     469.000   1.400',
 '5 -6     469.000   1.400',
 '1 -7     367.000   1.080',
 '2 -8     367.000   1.080',
 '3 -9     367.000   1.080',
 '4 -10    367.000   1.080',
 '5 -11    367.000   1.080',
 '6 -12    367.000   1.080',
 '',
 '']

In [134]:
new_bonds

['ca-ca     469.000  1.400',
 'ca-ca     469.000  1.400',
 'ca-ca     469.000  1.400',
 'ca-ca     469.000  1.400',
 'ca-ca     469.000  1.400',
 'ca-ca     469.000  1.400',
 'ca-ha     367.000  1.080',
 'ca-ha     367.000  1.080',
 'ca-ha     367.000  1.080',
 'ca-ha     367.000  1.080',
 'ca-ha     367.000  1.080',
 'ca-ha     367.000  1.080']

In [147]:
opening = "ANGLE"
between = "(.*?)"
closing = "DIHE"
pattern = opening + between + closing
angles = re.findall(pattern, smirnoff_frcmod, re.DOTALL)

angles_list = angles[0].split("\n")

new_angles = []
for angle in angles_list[1:-2]:
    atom_one, atom_two, atom_three, k, theta = angle.split()
    atom_two = atom_two.replace("-", "")
    atom_three = atom_three.replace("-", "")
    new_atom_one = dictionary[atom_one]
    new_atom_two = dictionary[atom_two]
    new_atom_three = dictionary[atom_three]
    new_angles.append(f"{new_atom_one:<2}-{new_atom_two:<2}-{new_atom_three:<2}{k:>11}{theta:>9}")

In [148]:
angles_list

['',
 '2 -1 -3      70.000  120.000',
 '1 -2 -4      70.000  120.000',
 '1 -3 -5      70.000  120.000',
 '2 -4 -6      70.000  120.000',
 '3 -5 -6      70.000  120.000',
 '4 -6 -5      70.000  120.000',
 '2 -1 -7      50.000  120.000',
 '3 -1 -7      50.000  120.000',
 '1 -2 -8      50.000  120.000',
 '4 -2 -8      50.000  120.000',
 '1 -3 -9      50.000  120.000',
 '5 -3 -9      50.000  120.000',
 '2 -4 -10     50.000  120.000',
 '6 -4 -10     50.000  120.000',
 '3 -5 -11     50.000  120.000',
 '6 -5 -11     50.000  120.000',
 '4 -6 -12     50.000  120.000',
 '5 -6 -12     50.000  120.000',
 '',
 '']

In [149]:
new_angles

['ca-ca-ca     70.000  120.000',
 'ca-ca-ca     70.000  120.000',
 'ca-ca-ca     70.000  120.000',
 'ca-ca-ca     70.000  120.000',
 'ca-ca-ca     70.000  120.000',
 'ca-ca-ca     70.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000',
 'ca-ca-ha     50.000  120.000']

In [255]:
opening = "DIHE"
between = "(.*?)"
closing = "IMPROPER"
pattern = opening + between + closing
dihedrals = re.findall(pattern, smirnoff_frcmod, re.DOTALL)

dihedrals_list = dihedrals[0].split("\n")

new_dihedrals = []
for dihedral in dihedrals_list[1:-2]:
    try:
        atom_one, atom_two, atom_three, atom_four, idiv, k, theta, per, scee, scnb = dihedral.split()
    except ValueError as e:
        if str(e) == "not enough values to unpack (expected 10, got 9)":
            atom_one, atom_two, atom_three, stuff = dihedral.split("-")
            atom_four, idiv, k, theta, per, scee, scnb = stuff.split()
                    
    atom_two = atom_two.replace("-", "")
    atom_three = atom_three.replace("-", "")
    atom_four = atom_four.replace("-", "")
    new_atom_one = dictionary[atom_one.strip()]
    new_atom_two = dictionary[atom_two.strip()]
    new_atom_three = dictionary[atom_three.strip()]
    new_atom_four = dictionary[atom_four.strip()]
    new_dihedrals.append(f"{new_atom_one:<2}-{new_atom_two:<2}-{new_atom_three:<2}-{new_atom_four:<2}{idiv:>5}{k:>15}{theta:>9}{per:>6}    {scee:<6} {scnb}")

Haven't done with impropers yet...

In [276]:
opening = "NONB"
forwards = ".*$"
pattern = opening + forwards
nonb = re.findall(pattern, smirnoff_frcmod, re.DOTALL)

nonb_list = nonb[0].split("\n")

new_nonb = []
for nonb in nonb_list[1:-2]:
    atom_type, sigma, epsilon = nonb.split()
    new_nonb.append(f"{dictionary[atom_type]}{sigma:>14}{epsilon:>14}")

In [277]:
nonb_list

['NONB',
 '1     1.90800000   0.08600000',
 '2     1.90800000   0.08600000',
 '3     1.90800000   0.08600000',
 '4     1.90800000   0.08600000',
 '5     1.90800000   0.08600000',
 '6     1.90800000   0.08600000',
 '7     1.45900000   0.01500000',
 '8     1.45900000   0.01500000',
 '9     1.45900000   0.01500000',
 '10    1.45900000   0.01500000',
 '11    1.45900000   0.01500000',
 '12    1.45900000   0.01500000',
 '',
 '']

In [278]:
new_nonb

['ca    1.90800000    0.08600000',
 'ca    1.90800000    0.08600000',
 'ca    1.90800000    0.08600000',
 'ca    1.90800000    0.08600000',
 'ca    1.90800000    0.08600000',
 'ca    1.90800000    0.08600000',
 'ha    1.45900000    0.01500000',
 'ha    1.45900000    0.01500000',
 'ha    1.45900000    0.01500000',
 'ha    1.45900000    0.01500000',
 'ha    1.45900000    0.01500000',
 'ha    1.45900000    0.01500000']

In [280]:
smirnoff_frcmod.split("\n")[0]

'Force field parameters from benzene.gaff-smirnoff.prmtop'